In [1]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import to_categorical

list_of_lists = []
with open('data\ecoli\Ecoli.txt') as f:
    for line in f:
        #inner_list = [elt.strip() for elt in line.split()]
        inner_list = list(line)
        list_of_lists.append(inner_list)
print(len(list_of_lists[0])) # About 4 MB
ecoli = list_of_lists[0]

n_symb = 4
s = ecoli
char_list = [97, 103, 99, 116] # we can read this as we go
print(char_list)
update_period = len(s)

k = 64 #context length
n = 100000 # train every
legend = dict([(v, k) for k, v in enumerate(char_list)]) # map character to 0,1,2,3,4, etc.
vocab_size = len(char_list)

temp_dict = {'a':97,'g': 103,'c': 99,'t': 116}
s = [temp_dict[i] for i in s]
# #Train model
# x = np.zeros((update_period-k, k, vocab_size, 1)) # 64 characters context
# y = np.zeros((update_period-k, vocab_size))


# print(len(s))
# idx3 = 0
# for idx2 in range(k,len(s)):
#     train_seq = [to_categorical(legend[i], num_classes=vocab_size ) for i in s[idx2-k:idx2]] 
#     train_target = to_categorical(legend[s[idx2]], num_classes=vocab_size )
#     x[idx3,:] = np.array(train_seq).reshape((k, vocab_size, 1))
#     y[idx3,:] = train_target
#     idx3 += 1

4638690
[97, 103, 99, 116]


In [ ]:
overall = 0
n_symb = 4
x = np.zeros((200000, k, n_symb, 1)) # 64 characters context
y = np.zeros((200000, n_symb))
print(len(s))
idx3 = 0
for idx2 in range(200000*overall+k,200000*(overall+1)+k): #len(s)):
    train_seq = [to_categorical(legend[i], num_classes=n_symb ) for i in s[idx2-k:idx2]] 
    train_target = to_categorical(legend[s[idx2]], num_classes=n_symb )
    x[idx3,:] = np.array(train_seq).reshape(k, n_symb, 1)
    y[idx3] = train_target
    idx3 += 1

## Sounds cool but my computer can't handle it

In [2]:
seq_length = 256  #Length of the sequence to be inserted into the LSTM
vocab_size = len(char_list)  #Size of the final dense layer of the model
lstm_cells = 352  #Size of the LSTM layer
from tensorflow.keras.layers import LayerNormalization

cnn_lstm = Sequential()
cnn_lstm.add(Conv2D(1, (2,2), activation='relu', padding='same', input_shape=(k, vocab_size,1)))
cnn_lstm.add(MaxPooling2D(pool_size=(2, 2)))
cnn_lstm.add(TimeDistributed(Flatten()))
cnn_lstm.add(LSTM(lstm_cells, return_sequences=True))
cnn_lstm.add(LayerNormalization(axis=1 , center=True , scale=True))
cnn_lstm.add(LSTM(lstm_cells, return_sequences=True))
cnn_lstm.add(LayerNormalization(axis=1 , center=True , scale=True))
cnn_lstm.add(LSTM(lstm_cells, return_sequences=True))
cnn_lstm.add(LayerNormalization(axis=1 , center=True , scale=True))
cnn_lstm.add(LSTM(lstm_cells))
cnn_lstm.add(LayerNormalization(axis=1 , center=True , scale=True))
cnn_lstm.add(Dense(vocab_size))
cnn_lstm.add(Activation('softmax'))
cnn_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# cnn_lstm.fit(x[:100000],y[:100000],
#                   batch_size=250,
#                   epochs=1,
#                   validation_data=(x[100000:],y[100000:]))
# cnn_lstm.summary()


In [ ]:
predicted_onehot = []
for i in range(len(ecoli)//n - 1):
    if i%5 == 0:
        print(i)
    cnn_lstm.fit(x[n*i:n*(i+1)], y[n*i:n*(i+1)], batch_size=256, epochs=2, verbose=1)

    predicted_onehot += list(clf.predict_proba(x[n*(i+1):n*(i+2)]))
predicted_onehot += list(clf.predict_proba(x[n*(len(ecoli)//n):]))

0


In [11]:
len(ecoli)//200000

23

In [12]:
len(ecoli)//500000

9

In [ ]:
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_CNNLSTM.txt'

e_idx = 0
tempdict = {}
#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:

    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs) # For the first 200,000
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!
    for symbol in s[:n+k]:
        t = model.get_total() ## New lines!
        l = model.get_low(legend[symbol])
        h = model.get_high(legend[symbol])
        enc.storeRegion(l,h,t) 
        model.increment(legend[symbol])
        e_idx += 1
        
    prior = [1/257 for i in range(257)]
    prior[256] = 1-sum(prior[:256])
#     model = ProbabilityList(prior)   # reset model
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs)
        
    
    for overall in range(24):
        predicted_val = []
        if overall <= 22:
            x = np.zeros((200000, k, n_symb, 1)) # 64 characters context
            y = np.zeros((200000, n_symb))
            print(len(s))
            idx3 = 0
            for idx2 in range(200000*overall+k,200000*(overall+1)+k): #len(s)):
                train_seq = [to_categorical(legend[i], num_classes=n_symb ) for i in s[idx2-k:idx2]]  
                train_target = legend[s[idx2]]
                x[idx3,:] = np.array(train_seq).reshape(k, n_symb, 1)
                y[idx3] = to_categorical(train_target, num_classes=n_symb )
                idx3 += 1
                
#             x = np.zeros((1000000, tsteps, seg_len)) # 64 characters context
#             y = np.zeros((1000000, n_symb))
#             print(len(s))
#             idx3 = 0
#             for idx2 in range(k,1000000+k): #len(s)):
#                 train_seq = [legend[i] for i in s[idx2-k:idx2]] 
#                 train_target = s[idx2]
#                 x[idx3,:] = np.array(train_seq).reshape(tsteps,seg_len)
#                 y[idx3] = to_categorical(train_target, num_classes=n_symb )
#                 idx3 += 1
                
        if overall == 23:
            x = np.zeros((len(s)-200000*overall-k, k, n_symb, 1)) # 64 characters context
            y = np.zeros((len(s)-200000*overall-k, n_symb))
            print(len(s))
            idx3 = 0
            for idx2 in range(200000*overall+k,len(s)): #len(s)):
                train_seq = [to_categorical(legend[i], num_classes=n_symb ) for i in s[idx2-k:idx2]] 
                train_target = legend[s[idx2]]
                x[idx3,:] = np.array(train_seq).reshape(k, n_symb, 1)
                y[idx3] = to_categorical(train_target, num_classes=n_symb )
                idx3 += 1

        if overall != 0:
            predicted_val += list(cnn_lstm.predict(x[0:n]))
            

        for i in range(1):
            cnn_lstm.fit(x[n*i:n*(i+1)], y[n*i:n*(i+1)],
                  batch_size=250,
                  epochs=2,
                  validation_data=(x[n*(i+1):], y[n*(i+1):]))

            predicted_val += list(cnn_lstm.predict(x[n*(i+1):n*(i+2)]))
#         i = len(x)//100000 -2
#         cnn_lstm.fit(x[n*i:n*(i+1)], y[n*i:n*(i+1)],
#                   batch_size=250,
#                   epochs=1,
#                   validation_data=(x[n*(i+1):], y[n*(i+1):]))
#         predicted_val += list(cnn_lstm.predict(x[n*(i+1):]))

        cnn_lstm.fit(x[n*(i+1):], y[n*(i+1):],
                  batch_size=250,
                  epochs=1)
        x= None
        y = None
        del x
        del y
        for prob_list in predicted_val:
            for val, prob in enumerate(prob_list):
                model.set(val, int(prob*100000)+1)
                
#             model.prob_list[:4] = prob_list
#             model.prob_list[4:256] = [1/100000 for i in range(252)]
#             model.normalize()
#             t = int(2**16) ## New lines!
#             l = int(model.get_low(legend[s[e_idx]])*2**16)
#             h = int(model.get_high(legend[s[e_idx]])*2**16)
#             enc.storeRegion(l,h,t) 
            t = model.get_total()
            l = model.get_low(legend[s[e_idx]])
            h = model.get_high(legend[s[e_idx]])
            enc.storeRegion(l,h,t) 
            e_idx += 1
        predicted_val = None
        del predicted_val

#     t = int(2**16) ## New lines!
#     l = int(model.get_low(256)*2**16)
#     h = int(model.get_high(256)*2**16)
    t = model.get_total()
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    
    enc.finish_encode(bitout)

4638690
Train on 100000 samples, validate on 100000 samples
Epoch 1/2
100000/100000 [==============================] - 54s 540us/sample - loss: 1.4246 - categorical_accuracy: 0.2678 - val_loss: 1.3850 - val_categorical_accuracy: 0.2529
Epoch 2/2
100000/100000 [==============================] - 39s 386us/sample - loss: 1.3839 - categorical_accuracy: 0.2828 - val_loss: 1.3778 - val_categorical_accuracy: 0.2927
Train on 100000 samples
100000/100000 [==============================] - 29s 289us/sample - loss: 1.3817 - categorical_accuracy: 0.2856
4638690
Train on 100000 samples, validate on 100000 samples
Epoch 1/2
100000/100000 [==============================] - 38s 384us/sample - loss: 1.3825 - categorical_accuracy: 0.2820 - val_loss: 1.3769 - val_categorical_accuracy: 0.2880
Epoch 2/2
100000/100000 [==============================] - 39s 385us/sample - loss: 1.3817 - categorical_accuracy: 0.2840 - val_loss: 1.3849 - val_categorical_accuracy: 0.2709
Train on 100000 samples
100000/100000 [=